In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import sys
import pandas as pd
import seaborn as sns
import os
from scipy.interpolate import interp1d
from scipy import interpolate
from pathlib import Path
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pyecharts import options as opts
from pyecharts.charts import Map, Timeline
from pyecharts.charts import Geo
import matplotlib.dates as mdates
import matplotlib.ticker as tick 
from matplotlib import rcParams
from datetime import datetime
import seaborn as sns
import geopandas as gpd
import folium
from pyecharts import options as opts
from pyecharts.faker import Faker
from sklearn.cluster import DBSCAN
from mpl_chord_diagram import chord_diagram
import matplotlib.colors as mcolors
from sklearn.linear_model import Ridge
from matplotlib.gridspec import GridSpec
from matplotlib import cm
from PIL import Image, ImageDraw, ImageFont
import folium
from shapely.geometry import Point
from pyproj import Geod
import webbrowser
import matplotlib.colors as mcolors
import xyzservices.providers as xyz

### Figure 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 设置数据路径和目标国家
data_path = '/burg/glab/users/wz2481/Power_project/MC_uncertainty/summarize'
sel_country = 'Netherlands'
file_path = os.path.join(data_path, f'{sel_country}.csv')

# 读取数据
df = pd.read_csv(file_path)
df = df.set_index(['Unnamed: 0'])
df.index = pd.to_datetime(df.index)

# 计算原始统计量
mean_series = df.mean(axis=1)
q05_series = df.quantile(0.025, axis=1)
q95_series = df.quantile(0.975, axis=1)

# === 计算7天滑动平均（包括不确定区间）===
days_num = 30
mean_smooth = mean_series.rolling(window=days_num, center=True).mean()
q05_smooth = q05_series.rolling(window=days_num, center=True).mean()
q95_smooth = q95_series.rolling(window=days_num, center=True).mean()

# 画图（两个子图）
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14, 8), sharex=True, sharey=True)

# 第一张图：原始均值和不确定区间
axes[0].plot(df.index, mean_series, label='Mean', color='black', linewidth=1)
axes[0].fill_between(df.index, q05_series, q95_series, color='gray', alpha=0.5, label='2.5–97.5% Quantile')
axes[0].set_ylabel('Load', fontsize=14)
axes[0].set_title(f'Simulated Load and Uncertainty Band – {sel_country}', fontsize=18, weight='bold')
axes[0].legend(fontsize=12)
axes[0].tick_params(axis='both', which='major', labelsize=12, length=8, width=1.5, direction='out')
axes[0].grid(False)
for spine in axes[0].spines.values():
    spine.set_linewidth(2)

# 第二张图：7天滑动平均及其不确定区间
axes[1].plot(df.index, mean_smooth, label=f'{days_num}-day Moving Average', color='black', linewidth=2)
axes[1].fill_between(df.index, q05_smooth, q95_smooth, color='gray', alpha=0.3, label='Smoothed 2.5–97.5% Quantile')
axes[1].set_xlabel('Date', fontsize=14)
axes[1].set_ylabel('Smoothed Load', fontsize=14)
axes[1].legend(fontsize=12)
axes[1].tick_params(axis='both', which='major', labelsize=12, length=8, width=1.5, direction='out')
axes[1].grid(False)
for spine in axes[1].spines.values():
    spine.set_linewidth(2)

plt.tight_layout()
plt.show()

### Figure 3

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# ===== 国家缩写字典 =====
country_abbrev = {
    'Austria': 'AT', 'Belgium': 'BE', 'Bulgaria': 'BG', 'Croatia': 'HR',
    'Czech Republic': 'CZ', 'Denmark': 'DK', 'Estonia': 'EE', 'Finland': 'FI',
    'France': 'FR', 'Germany': 'DE', 'Greece': 'GR', 'Hungary': 'HU',
    'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Montenegro': 'ME',
    'Netherlands': 'NL', 'North Macedonia': 'MK', 'Poland': 'PL', 'Slovenia': 'SI',
    'Spain': 'ES', 'Sweden': 'SE', 'Portugal': 'PT', 'United Kingdom': 'UK',
    'Bosnia and Herz.': 'BA', 'Georgia': 'GE', 'Kosovo': 'XK', 'Latvia': 'LV',
    'Luxembourg': 'LU', 'Moldova': 'MD', 'Norway': 'NO', 'Romania': 'RO',
    'Serbia': 'RS', 'Slovakia': 'SK', 'Switzerland': 'CH'
}

# ===== 读取数据 =====
df_econ = pd.read_csv('.\Main_data_2015_2024_real0.csv', parse_dates=['date'])
df_monthly = pd.read_csv('.\EM_country_monthly_averages_con.csv')

# ===== 处理日期并聚合 df_econ =====
df_econ['month'] = df_econ['date'].dt.to_period('M').astype(str)
econ_monthly = df_econ.groupby(['country', 'month'], as_index=False).mean(numeric_only=True)

# ===== 标准化月份格式并转换长表 =====
df_monthly = df_monthly.rename(columns={df_monthly.columns[0]: 'month'})
df_monthly['month'] = pd.to_datetime(df_monthly['month'], format='%b-%y').dt.to_period('M').astype(str)
df_monthly_long = df_monthly.melt(id_vars='month', var_name='country', value_name='EM_con')

# ===== 合并并添加 month_name =====
df_merged = pd.merge(econ_monthly, df_monthly_long, on=['country', 'month'], how='inner')
df_merged['month_name'] = pd.to_datetime(df_merged['month']).dt.month_name()

# ===== 计算中位数和 IQR（25–75分位数） =====
def iqr_bounds(group):
    return pd.Series({
        'consumption_CI': group['consumption_CI'].median(),
        'EM_con': group['EM_con'].median(),
        'consumption_CI_err_low': group['consumption_CI'].quantile(0.25),
        'consumption_CI_err_high': group['consumption_CI'].quantile(0.75),
        'EM_con_err_low': group['EM_con'].quantile(0.25),
        'EM_con_err_high': group['EM_con'].quantile(0.75),
    })

df_plot = df_merged.groupby(['month_name', 'country']).apply(iqr_bounds).reset_index()

# ===== 排序月份顺序 =====
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
df_plot['month_name'] = pd.Categorical(df_plot['month_name'], categories=month_order, ordered=True)
df_plot = df_plot.sort_values('month_name')

# ===== 坐标轴范围 =====
lims = [0, 1000]

from adjustText import adjust_text

##========绘图方式3==========
import seaborn as sns
from adjustText import adjust_text

# 设置风格和字体
sns.set_style("white")
sns.set_context("talk", font_scale=1.5)  # 更大字体

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(20, 24))
axes = axes.flatten()

palette = sns.color_palette("deep")
point_color = palette[0]
error_color = "lightblue"

for idx, month in enumerate(month_order):
    ax = axes[idx]
    df_m = df_plot[df_plot['month_name'] == month].dropna()

    if df_m.empty:
        ax.set_visible(False)
        continue

    # 统一绘制所有国家（使用圆点、无标签）
    ax.errorbar(df_m['consumption_CI'], df_m['EM_con'],
                xerr=[df_m['consumption_CI'] - df_m['consumption_CI_err_low'],
                      df_m['consumption_CI_err_high'] - df_m['consumption_CI']],
                yerr=[df_m['EM_con'] - df_m['EM_con_err_low'],
                      df_m['EM_con_err_high'] - df_m['EM_con']],
                fmt='o', color=point_color, ecolor=error_color,
                elinewidth=1.5, capsize=4, alpha=0.9,
                markersize=10, markeredgewidth=1, markeredgecolor='black')

    # 添加 R²
    x = df_m['consumption_CI']
    y = df_m['EM_con']
    if len(x) >= 2:
        r = np.corrcoef(x, y)[0, 1]
        r2 = r ** 2
        ax.text(0.03, 0.97, f'{month}\n$R^2$ = {r2:.2f}',
                transform=ax.transAxes, fontsize=20, weight='bold',
                ha='left', va='top', color='black')

    # 坐标轴设置
    ax.set_xlabel(r"ECON-PowerCI ($\mathrm{tCO_2eq/GWh}$)", fontsize=20)
    ax.set_ylabel(r"Electricity Map ($\mathrm{tCO_2eq/GWh}$)", fontsize=20)
    ax.tick_params(axis='both', labelsize=20)
    ax.plot(lims, lims, linestyle='--', color='gray', linewidth=1.2)
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_aspect('equal', adjustable='box')

    # 加黑边框
    for spine in ax.spines.values():
        spine.set_color('black')
        spine.set_linewidth(3)

plt.subplots_adjust(hspace=0.25, wspace=0.15)
plt.tight_layout()
plt.show()



In [ ]:
# === 国家简称映射 ===
country_abbrev = {
    'Austria': 'AT', 'Belgium': 'BE', 'Bulgaria': 'BG', 'Croatia': 'HR',
    'Czech Republic': 'CZ', 'Denmark': 'DK', 'Estonia': 'EE', 'Finland': 'FI',
    'France': 'FR', 'Germany': 'DE', 'Greece': 'GR', 'Hungary': 'HU',
    'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Montenegro': 'ME',
    'Netherlands': 'NL', 'North Macedonia': 'MK', 'Poland': 'PL', 'Slovenia': 'SI',
    'Spain': 'ES', 'Sweden': 'SE', 'Portugal': 'PT', 'United Kingdom': 'UK',
    'Bosnia and Herz.': 'BA', 'Georgia': 'GE', 'Kosovo': 'XK', 'Latvia': 'LV',
    'Luxembourg': 'LU', 'Moldova': 'MD', 'Norway': 'NO', 'Romania': 'RO',
    'Serbia': 'RS', 'Slovakia': 'SK', 'Switzerland': 'CH'
}

# === 指定需标注简称的国家 ===
highlighted_countries = {'DE', 'FR', 'IT', 'SE', 'PL', 'NL', 'UK'}

# === 加载数据 ===
df_yearly = pd.read_csv('.EM_country_yearly_averages_con.csv')

# df_econ 需要你事先加载，并包含 'date' 列
df_econ['year'] = df_econ['date'].dt.year
econ_yearly = df_econ.groupby(['country', 'year'], as_index=False).mean(numeric_only=True)

# === 处理长格式 ===
df_yearly_long = df_yearly.dropna(subset=['Country'])
df_yearly_long = df_yearly_long.melt(id_vars='Country', var_name='year', value_name='EM_con')
df_yearly_long = df_yearly_long.rename(columns={'Country': 'country'})
df_yearly_long['year'] = df_yearly_long['year'].astype(int)

# === 合并数据 ===
df_merged_yearly = pd.merge(econ_yearly, df_yearly_long, on=['country', 'year'], how='inner')
df_merged_yearly = df_merged_yearly.sort_values(['country', 'year'])

# === 计算均值和标准差 ===
df_mean = df_merged_yearly.groupby('country', as_index=False)[['consumption_CI', 'EM_con']].mean()
df_std = df_merged_yearly.groupby('country', as_index=False)[['consumption_CI', 'EM_con']].std()

df_plot = df_mean.copy()
df_plot['consumption_CI_std'] = df_std['consumption_CI']
df_plot['EM_con_std'] = df_std['EM_con']
df_plot['abbrev'] = df_plot['country'].map(lambda c: country_abbrev.get(c, c))

# # === 拆分要标注和不标注的国家 ===
# df_highlight = df_plot[df_plot['abbrev'].isin(highlighted_countries)]
# df_other = df_plot[~df_plot['abbrev'].isin(highlighted_countries)]

# === 回归分析 ===
X = df_plot[['consumption_CI']].values
y = df_plot['EM_con'].values
model = LinearRegression().fit(X, y)
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

# === 绘图开始 ===
fig, ax = plt.subplots(figsize=(7, 7))
lims = [0, 1000]

# === 所有国家统一样式 ===
ax.errorbar(df_plot['consumption_CI'], df_plot['EM_con'],
            xerr=df_plot['consumption_CI_std'],
            yerr=df_plot['EM_con_std'],
            fmt='o', color='tab:blue', ecolor='lightblue',
            elinewidth=1.2, capsize=4, markersize=8,
            markeredgecolor='black', markeredgewidth=1, alpha=0.9)


# === 添加1:1参考线 ===
ax.plot(lims, lims, color='gray', linestyle='--', linewidth=1.2)

# === 添加 R² 值 ===
ax.text(0.03, 0.97, f'$R^2$ = {r2:.2f}',
        transform=ax.transAxes, fontsize=18, weight='bold',
        verticalalignment='top', horizontalalignment='left')

# === 坐标轴设置 ===
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_aspect('equal', adjustable='box')
ax.set_xlabel(r"ECON-PowerCI ($\mathrm{tCO_2eq/GWh}$)", fontsize=16)
ax.set_ylabel(r"Electricity Map ($\mathrm{tCO_2eq/GWh}$)", fontsize=16)
ax.tick_params(axis='both', labelsize=16)

# === 边框加粗 ===
for spine in ax.spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(1.5)

plt.tight_layout()
plt.show()


### Figure 4

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define a function to filter data within the 1-99 percentiles
def filter_percentile(df, column, lower=0.01, upper=0.99):
    lower_bound = df[column].quantile(lower)
    upper_bound = df[column].quantile(upper)
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


# Load the data
data = pd.read_csv('./Main_data_2015_2024_real0.csv')

# Country abbreviations for two-letter codes
country_abbrev = {
    'Austria': 'AT',
    'Belgium': 'BE',
    'Bulgaria': 'BG',
    'Croatia': 'HR',
    'Czech Republic': 'CZ',
    'Denmark': 'DK',
    'Estonia': 'EE',
    'Finland': 'FI',
    'France': 'FR',
    'Germany': 'DE',
    'Greece': 'GR',
    'Hungary': 'HU',
    'Ireland': 'IE',
    'Italy': 'IT',
    'Lithuania': 'LT',
    'Montenegro': 'ME',
    'Netherlands': 'NL',
    'North Macedonia': 'MK',
    'Poland': 'PL',
    'Slovenia': 'SI',
    'Spain': 'ES',
    'Sweden': 'SE',
    'Portugal': 'PT',
    'United Kingdom': 'UK'
}

# Filter data for selected countries, including only the countries specified in country_sel
country_sel = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Lithuania', 'Montenegro', 'Netherlands', 'North Macedonia', 'Poland', 'Slovenia', 'Spain', 'Sweden', 'Portugal', 'United Kingdom']

data = data[data['country'].isin(country_sel)]

# Country-specific filtering
constraints = {
    'Croatia': ('2019-01', None),
    'Georgia': ('2022-01', None),
    'Italy': ('2016-01', None),
    'Kosovo': ('2022-01', None),
    'Luxembourg': ('2022-01', None),
    'Moldova': ('2022-01', None),
    'Norway': ('2017-01', None),
    'Serbia': ('2017-01', None),
    'Spain': ('2017-01', None),
    'United Kingdom': ('2016-01', '2021-01')
}

# List of countries to delete
countries_to_delete = ['Bosnia and Herz', 'Cyprus', 'Ukraine']

# Filter out unwanted countries
data = data[~data['country'].isin(countries_to_delete)]

# Apply country-specific date constraints
for country, (start_date, end_date) in constraints.items():
    if start_date and end_date:
        data = data[(data['country'] != country) | ((data['date'] >= start_date) & (data['date'] <= end_date))]
    elif start_date:
        data = data[(data['country'] != country) | (data['date'] >= start_date)]

# Dictionary to store the yearly differences for each country
yearly_diff_dict = {}

filtered_data = pd.DataFrame()
outlier_data = pd.DataFrame()  # 新增：存储所有国家的 outliers

for country in data['country'].unique():
    country_data = data[data['country'] == country]

    # 提取 generation_CI 异常值
    gen_q1 = country_data['generation_CI'].quantile(0.01)
    gen_q99 = country_data['generation_CI'].quantile(0.99)
    gen_outliers = country_data[(country_data['generation_CI'] < gen_q1) | (country_data['generation_CI'] > gen_q99)]
    gen_outliers = gen_outliers[['date', 'generation_CI']].copy()
    gen_outliers['type'] = 'generation_CI'
    gen_outliers = gen_outliers.rename(columns={'generation_CI': 'value'})
    gen_outliers['country'] = country

    # 提取 consumption_CI 异常值
    con_q1 = country_data['consumption_CI'].quantile(0.01)
    con_q99 = country_data['consumption_CI'].quantile(0.99)
    con_outliers = country_data[(country_data['consumption_CI'] < con_q1) | (country_data['consumption_CI'] > con_q99)]
    con_outliers = con_outliers[['date', 'consumption_CI']].copy()
    con_outliers['type'] = 'consumption_CI'
    con_outliers = con_outliers.rename(columns={'consumption_CI': 'value'})
    con_outliers['country'] = country

    # 合并两类 outliers，添加到总 outlier_data 中
    outlier_data = pd.concat([outlier_data, gen_outliers, con_outliers], ignore_index=True)

    # Filter percentiles and build filtered_data as原本逻辑
    filtered_country_data = filter_percentile(country_data, 'generation_CI')
    filtered_country_data = filter_percentile(filtered_country_data, 'consumption_CI')

    filtered_data = pd.concat([filtered_data, filtered_country_data])

    # Yearly stats（略）
    filtered_country_data['date'] = pd.to_datetime(filtered_country_data['date'])
    yearly_means = filtered_country_data.groupby(filtered_country_data['date'].dt.year)[['consumption_CI', 'generation_CI']].mean()
    yearly_diff = yearly_means['consumption_CI'] - yearly_means['generation_CI']
    yearly_diff_dict[country] = yearly_diff


# Now use `filtered_data` for plotting instead of `data`
data = filtered_data

# Set up for plotting all countries in one figure with 4x3 grid
sns.set_style('white')
palette = 'Set2'

# Set global font size and remove right/top spines
plt.rcParams.update({
    'font.size': 20,         # Global font size
    'axes.titlesize': 24,    # Title font size
    'axes.labelsize': 22,    # Axis labels font size
    'xtick.labelsize': 18,   # X-axis tick labels font size
    'ytick.labelsize': 18,   # Y-axis tick labels font size
    'legend.fontsize': 20,   # Legend font size
    'figure.titlesize': 26   # Figure title font size
})

# Create plots for every 12 countries in separate figures
for batch_start in range(0, len(country_sel), 12):
    countries_batch = country_sel[batch_start:batch_start + 12]
    
    # Create the figure and axes for subplots
    n_countries = len(countries_batch)
    fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 14))
    axes = axes.flatten()  # Flatten the axes array for easier indexing

    # Loop through each country in the batch and assign its plot to the corresponding subplot
    for i, country in enumerate(countries_batch):
        country_data = data[data['country'] == country]

        # Convert the 'date' column to a datetime object for grouping by year
        country_data['date'] = pd.to_datetime(country_data['date'])
        
        # Calculate yearly means for consumption_CI and generation_CI
        yearly_means = country_data.groupby(country_data['date'].dt.year)[['consumption_CI', 'generation_CI']].mean()
        
        # Calculate the yearly differences (average consumption_CI - average generation_CI)
        yearly_diff = yearly_means['consumption_CI'] - yearly_means['generation_CI']
        
        # Calculate the mean, 25th, and 75th percentiles of the yearly differences
        mean_diff = yearly_diff.mean()
        q25_diff = yearly_diff.quantile(0.25)
        q75_diff = yearly_diff.quantile(0.75)

        # Melt the data for easier plotting (converting generation_CI and consumption_CI into a long format)
        melted_data = pd.melt(country_data, id_vars=["date"], value_vars=["generation_CI", "consumption_CI"],
                            var_name="type", value_name="value")
        
        ax = axes[i]  # Select the appropriate subplot

        # Plot violin plot with cut=0 to restrict the violin plot to the data range
        sns.violinplot(x="type", y="value", data=melted_data, palette=palette, scale="width", inner=None, ax=ax, cut=0)
        
        # Get limits before clipping the violin
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        
        # Clip each violin to only show half
        for violin in ax.collections:
            bbox = violin.get_paths()[0].get_extents()
            x0, y0, width, height = bbox.bounds
            violin.set_clip_path(plt.Rectangle((x0, y0), width / 2, height, transform=ax.transData))
        
        # Add boxplot
        sns.boxplot(x="type", y="value", data=melted_data, saturation=1, showfliers=False,
                    width=0.3, boxprops={'zorder': 3, 'facecolor': 'none'}, ax=ax)
        
        # Add scatter points (stripplot) with smaller point size
        old_len_collections = len(ax.collections)
        # sns.stripplot(x="type", y="value", data=melted_data, palette=palette, dodge=False, ax=ax, size=2)  # Reduced point size
        # 分别绘制非异常值（彩色）和异常值（黑色）
        
        custom_palette = {
            'generation_CI': '#66c2a5',  # 对应 Set2 的第一个颜色
            'consumption_CI': '#fc8d62'  # 对应 Set2 的第二个颜色
        }
        
        # 先绘制非离群点（用调色板颜色）
        for var in ['generation_CI', 'consumption_CI']:
            subset = melted_data[melted_data['type'] == var]
            q1 = subset['value'].quantile(0.25)
            q3 = subset['value'].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr

            non_outliers = subset[(subset['value'] >= lower_bound) & (subset['value'] <= upper_bound)]
            sns.stripplot(
                x='type', y='value', data=non_outliers,
                color=custom_palette[var], dodge=False, ax=ax, size=2, jitter=True, zorder=4
            )

        # 再绘制离群点（用黑色，保持 stripplot 的 jitter 和样式）
        for var in ['generation_CI', 'consumption_CI']:
            subset = melted_data[melted_data['type'] == var]
            q1 = subset['value'].quantile(0.25)
            q3 = subset['value'].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr

            outliers = subset[(subset['value'] < lower_bound) | (subset['value'] > upper_bound)]
            sns.stripplot(
                x='type', y='value', data=outliers,
                color='black', dodge=False, ax=ax, size=2, jitter=True, zorder=5
            )

        # Adjust scatter points to avoid overlap
        for dots in ax.collections[old_len_collections:]:
            dots.set_offsets(dots.get_offsets() + np.array([0.12, 0]))
        
        # Set title and remove legend
        ax.set_title(f'{country}')
        if ax.legend_:
            ax.legend_.remove()
        
        # Thicken left and bottom spines (for main plot), and remove right and top
        ax.spines['left'].set_linewidth(2.5)
        ax.spines['bottom'].set_linewidth(2.5)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

        # Adjust the ticks (make y-ticks thicker and point outward)
        ax.tick_params(axis='y', direction='out', width=4)
        ax.tick_params(axis='x', width=2.5)

        # Extend x-axis a bit to the right to make room for the inset
        xlim = ax.get_xlim()
        ax.set_xlim(xlim[0], xlim[1] + 0.5)

        # Remove x-axis label (type label)
        ax.set_xlabel('')

        # Change x-axis labels to 'gen_CI' and 'con_CI'
        ax.set_xticklabels(['gen_CI', 'con_CI'])

        # Set y-axis label to "tCO2eq/GWh" for the main plot
        ax.set_ylabel('tCO2eq/GWh')

        # Add inset axes for generation_CI - consumption_CI boxplot
        # inset_ax = ax.inset_axes([0.93, 0.53, 0.12, 0.54], facecolor='none')  # Adjust the position to top-right corner
        inset_ax = ax.inset_axes([0.93, 0.23, 0.12, 0.94], facecolor='none')  # Adjust the position to top-right corner

        # Add a blue scatter point for the mean difference
        inset_ax.scatter([0], [mean_diff], color='blue', s=600, zorder=5)  # Larger blue point

        # Add whiskers for the 25th and 75th percentiles
        inset_ax.vlines(0, q25_diff, q75_diff, color='black', linestyle='-', linewidth=2.5, zorder=4)  # IQR whiskers

        # Add country abbreviation in white on top of the blue marker
        inset_ax.text(0, mean_diff, country_abbrev[country], color='white', fontsize=16, ha='center', va='center', zorder=6)

        # Add horizontal lines at the 25th and 75th percentiles (whisker tops)
        inset_ax.hlines(q25_diff, -0.1, 0.1, color='black', linewidth=2.5, zorder=5)  # 25th percentile line
        inset_ax.hlines(q75_diff, -0.1, 0.1, color='black', linewidth=2.5, zorder=5)  # 75th percentile line
        
        # # Add blue borders (spines) around the inset axes
        # for spine in inset_ax.spines.values():
        #     spine.set_edgecolor('blue')  # Set the edge color to blue
        #     spine.set_linewidth(2.5)     # Set the border width to 2.5


        # Only keep the right and top spines for the inset
        inset_ax.spines['left'].set_visible(False)
        inset_ax.spines['bottom'].set_visible(False)
        inset_ax.spines['top'].set_visible(False)
        inset_ax.spines['right'].set_linewidth(2.5)

        # Add a dashed horizontal line at y=0 in the inset
        inset_ax.axhline(y=0, color='black', linestyle='--', linewidth=2)

        # Adjust y-limits to accommodate whiskers (IQR range)
        lower_limit = min(q25_diff * 1.2, -abs(mean_diff) * 1.2)
        upper_limit = max(q75_diff * 1.2, abs(mean_diff) * 1.2)
        inset_ax.set_xlim(-0.25, 0.25)
        inset_ax.set_ylim(lower_limit, upper_limit)  # Symmetric limits around 0

        # Display y-axis ticks on the right in the inset, remove left ticks, increase font size and thickness
        inset_ax.tick_params(axis='y', labelsize=18, width=2.5, right=True, labelright=True, left=False, labelleft=False)
        inset_ax.set_xticks([])  # Remove x-axis ticks
        inset_ax.set_xlabel('')

        # Set y-axis label for the inset to "tCO2eq/GWh" and move it outside the right edge
        # inset_ax.set_ylabel('tCO2eq/GWh', fontsize=12, labelpad=20)
        inset_ax.yaxis.set_label_position('right')

    # Adjust layout and show plot
    plt.tight_layout()
    plt.show()

    # Close the figure to free up memory
    plt.close(fig)

# The dictionary `yearly_diff_dict` now stores the yearly differences for each country.

### Figure 6

In [ ]:
## Figure 4

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 加载数据
# data = pd.read_csv('./Main_data_2015_2023_new.csv')
data = pd.read_csv('.\Main_data_2015_2024_real0.csv')

# 国家缩写字典
country_abbrev = {
    'Austria': 'AT', 'Belgium': 'BE', 'Bulgaria': 'BG', 'Croatia': 'HR', 'Czech Republic': 'CZ',
    'Denmark': 'DK', 'Estonia': 'EE', 'Finland': 'FI', 'France': 'FR', 'Germany': 'DE', 'Greece': 'GR',
    'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Montenegro': 'ME', 
    'Netherlands': 'NL', 'North Macedonia': 'MK', 'Poland': 'PL', 'Slovenia': 'SI', 'Spain': 'ES',
    'Sweden': 'SE', 'Portugal': 'PT', 'United Kingdom': 'UK'
}

# 筛选的国家
country_sel = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 
               'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Lithuania', 'Montenegro', 
               'Netherlands', 'North Macedonia', 'Poland', 'Slovenia', 'Spain', 'Sweden', 'Portugal', 
               'United Kingdom']

# 使用较深的调色板
colors = sns.color_palette("dark", len(country_sel))  # 使用更深的色调

# 设置绘图风格
sns.set_style('white')
plt.rcParams.update({
    'font.size': 18,
    'axes.titlesize': 20,
    'axes.labelsize': 18,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 16,
    'axes.linewidth': 2  # 增加子图边框粗细
})

# 定义筛选 1-99 分位数并填充 NaN 的函数
def filter_percentile(df, column, lower=0.01, upper=0.99):
    lower_bound = df[column].quantile(lower)
    upper_bound = df[column].quantile(upper)
    df[column] = np.where((df[column] >= lower_bound) & (df[column] <= upper_bound), df[column], np.nan)
    return df

# 统计百分比变化
stas_arr = []

# 创建 8 行 3 列的子图
fig, axes = plt.subplots(8, 3, figsize=(18, 28.38))
axes = axes.flatten()  # 扁平化数组，以便可以通过索引访问

# a,b,c,d,...标签
subplot_labels = list('abcdefghijklmnopqrstuvwxyz')

# 对每个国家进行绘图
for i, country in enumerate(country_sel):
    ax = axes[i]
    country_data = data[data['country'] == country]

    # 将日期转换为 datetime
    country_data['date'] = pd.to_datetime(country_data['date'])

    # 筛选 1-99 分位数并填充 NaN
    country_data = filter_percentile(country_data, 'consumption_CI')

    # 初始化保存数据的 DataFrame
    plot_data = {}

    # 筛选2019和2020年的数据
    for year in [2019, 2020]:
        year_data = country_data[country_data['date'].dt.year == year]

        # 计算 DOY
        year_data['DOY'] = year_data['date'].dt.dayofyear

        # 对NaN值使用滑动平均进行插值
        year_data['rolling_avg'] = year_data['consumption_CI'].interpolate(method='linear').rolling(window=7, min_periods=1, center=True).mean()

        # 修正非闰年数据
        if len(year_data) < 366:
            last_doy = year_data['DOY'].max()
            if last_doy < 366:
                additional_data = pd.DataFrame({'DOY': [366], 'rolling_avg': [np.nan]})
                year_data = pd.concat([year_data, additional_data], ignore_index=True)

        plot_data[year] = year_data[['DOY', 'rolling_avg']].set_index('DOY')

    # 计算前六个月（1-6月）2020 vs 2019 的百分比变化
    if 2019 in plot_data and 2020 in plot_data:
        first_six_months_2019 = plot_data[2019].loc[:181, 'rolling_avg'].mean()
        first_six_months_2020 = plot_data[2020].loc[:181, 'rolling_avg'].mean()
        
        if first_six_months_2019 != 0:  # 防止除零错误
            percentage_change = ((first_six_months_2020 - first_six_months_2019) / first_six_months_2019) * 100
        else:
            percentage_change = np.nan

        diff_mean = first_six_months_2020 - first_six_months_2019
        stas_arr.append([country, percentage_change, first_six_months_2019, first_six_months_2020, diff_mean])

        # 绘制2019和2020年的数据，并填充它们之间的区域
        for year, df in plot_data.items():
            ax.plot(df.index, df['rolling_avg'], label=f'{year}', color=colors[i], linestyle='-' if year == 2020 else '--', linewidth=2.5 if year == 2020 else 1.5)

        ax.fill_between(plot_data[2019].index, plot_data[2019]['rolling_avg'], plot_data[2020]['rolling_avg'], color=colors[i], alpha=0.3)

        # 在子图的中央上方显示百分比变化，颜色与线条一致
        ax.text(0.5, 0.9, f'{percentage_change:.1f}%', transform=ax.transAxes, ha='center', va='center', fontsize=18, fontweight='bold', color=colors[i])

        # # 添加2019和2020的文本，使用对应颜色，放在图形上方或下方
        # ax.text(320, plot_data[2019].loc[300, 'rolling_avg'] + 5, '2019', color=colors[i], fontsize=12, fontweight='bold')
        # ax.text(320, plot_data[2020].loc[300, 'rolling_avg'] - 5, '2020', color=colors[i], fontsize=12, fontweight='bold')

    # 添加1月1日到6月30日之间的灰色阴影区域
    ax.axvspan(0, 181, color='gray', alpha=0.3)  # DOY 1到181（6月30日是第181天）
    ax.set_xlim([0, 366])

    # 设置横坐标显示为月份
    ax.set_xticks([1, 60, 121, 182, 244, 305])
    ax.set_xticklabels(['1-Jan', '1-Mar', '1-May', '1-Jul', '1-Sep', '1-Nov'])

    # 设置图例和标题
    ax.set_title(f'{country}', color='black', fontsize=20, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('tCO2eq/GWh')

    # 增加边框厚度，并设置纯黑色
    for spine in ax.spines.values():
        spine.set_linewidth(2)
        spine.set_color('black')

    # 设置刻度线朝外，字体颜色为黑色并加粗
    ax.tick_params(axis='both', direction='out', width=2, colors='black', labelsize=16)
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')

    # 添加子图标签 a, b, c, ...
    ax.text(0.02, 0.15, subplot_labels[i], transform=ax.transAxes, fontsize=20, fontweight='bold', color='black', ha='left', va='top')

    # 添加图例，没有外框，横向放置，并使用对应线条的颜色
    legend_labels = ['2019', '2020']
    handles = [plt.Line2D([0], [0], color=colors[i], linestyle='--', lw=2, label='2019'),
               plt.Line2D([0], [0], color=colors[i], linestyle='-', lw=2.5, label='2020')]
    
    # legend = ax.legend(handles=handles, loc='best', ncol=2, frameon=False)
    
    # # 修改 legend 字体颜色为对应的线条颜色
    # for text, handle in zip(legend.get_texts(), handles):
    #     text.set_color(handle.get_color())

# 移除多余的子图框架（适用于少于24个国家的情况）
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# 调整子图布局
plt.tight_layout()
plt.show()

### Figure 7

In [ ]:
import warnings
warnings.filterwarnings("ignore")


# 加载数据
# data = pd.read_csv('./Main_data_2015_2023_new.csv')

data = pd.read_csv('.\Main_data_2015_2024_real0.csv')

# 要显示的国家列表
selected_countries = ['Austria', 'Czech Republic', 'Estonia', 'France', 'Germany', 'Hungary',
                      'Ireland', 'Italy', 'Lithuania', 'Poland', 'Slovakia', 'Romania']

# 设置图形布局
rows = 6
cols = 2

# 绘图颜色和样式
colors = {'2021': 'black', '2022': 'black'}
linestyles = {'2021': '--', '2022': '-'}

# 设置绘图风格
sns.set_style('white')
plt.rcParams.update({
    'font.size': 22,
    'axes.titlesize': 22,
    'axes.labelsize': 22,
    'xtick.labelsize': 22,
    'ytick.labelsize': 22,
    'legend.fontsize': 22,
    'axes.linewidth': 2
})

# 定义筛选 1-99 分位数并填充 NaN 的函数
def filter_percentile(df, column, lower=0.01, upper=0.99):
    lower_bound = df[column].quantile(lower)
    upper_bound = df[column].quantile(upper)
    df[column] = np.where((df[column] >= lower_bound) & (df[column] <= upper_bound), df[column], np.nan)
    return df

war_stats_arr = []

# 创建子图
fig, axes = plt.subplots(rows, cols, figsize=(18, rows * 4))
axes = axes.flatten()

# a,b,c,d,... 标签
subplot_labels = [chr(97 + i) for i in range(len(selected_countries))]

# 对每个国家进行绘图
for i, country in enumerate(selected_countries):
    ax = axes[i]
    country_data = data[data['country'] == country]

    # 将日期转换为 datetime
    country_data['date'] = pd.to_datetime(country_data['date'])

    # 筛选 1-99 分位数并填充 NaN
    country_data = filter_percentile(country_data, 'consumption_CI')

    # 初始化保存数据的 DataFrame
    plot_data = {}

    # 筛选2021和2022年的数据
    for year in [2021, 2022]:
        year_data = country_data[country_data['date'].dt.year == year]

        # 计算 DOY
        year_data['DOY'] = year_data['date'].dt.dayofyear

        # 只保留1月到4月的数据（DOY 1到120）
        year_data = year_data[year_data['DOY'] <= 120]

        # 对NaN值使用滑动平均进行插值
        year_data['rolling_avg'] = year_data['consumption_CI'].interpolate(method='linear').rolling(window=7, min_periods=1, center=True).mean()

        plot_data[year] = year_data[['DOY', 'rolling_avg']].set_index('DOY')

    # 绘制2021和2022年的数据，并填充它们之间的区域
    if 2021 in plot_data and 2022 in plot_data:
        # 2022年高于2021年填充为红色，低于2021年填充为蓝色
        ax.fill_between(plot_data[2021].index, plot_data[2021]['rolling_avg'], plot_data[2022]['rolling_avg'], 
                        where=(plot_data[2022]['rolling_avg'] > plot_data[2021]['rolling_avg']), color='red', alpha=0.3)
        ax.fill_between(plot_data[2021].index, plot_data[2021]['rolling_avg'], plot_data[2022]['rolling_avg'], 
                        where=(plot_data[2022]['rolling_avg'] <= plot_data[2021]['rolling_avg']), color='blue', alpha=0.3)

        for year, df in plot_data.items():
            ax.plot(df.index, df['rolling_avg'], label=f'{year}', color=colors[str(year)], linestyle=linestyles[str(year)], linewidth=2.5)

        # 计算55-90 DOY的百分比变化
        period1_2021 = plot_data[2021]['rolling_avg'][55:90].mean()
        period1_2022 = plot_data[2022]['rolling_avg'][55:90].mean()
        if period1_2021 != 0:
            change1 = ((period1_2022 - period1_2021) / period1_2021) * 100
        else:
            change1 = np.nan

        # 计算90-120 DOY的百分比变化
        period2_2021 = plot_data[2021]['rolling_avg'][90:120].mean()
        period2_2022 = plot_data[2022]['rolling_avg'][90:120].mean()
        if period2_2021 != 0:
            change2 = ((period2_2022 - period2_2021) / period2_2021) * 100
        else:
            change2 = np.nan

        diff_mean_1 = period1_2022 - period1_2021
        diff_mean_2 = period2_2022 - period2_2021

        war_stats_arr.append([country, change1, change2, diff_mean_1, diff_mean_2, period1_2021, period1_2022, period2_2021, period2_2022])
                              
        # 在对应位置显示百分比变化
        ax.text(0.6, 0.9, f'{change1:.1f}%', transform=ax.transAxes, ha='center', va='center', fontsize=18, fontweight='bold', color='black')
        ax.text(0.85, 0.9, f'{change2:.1f}%', transform=ax.transAxes, ha='center', va='center', fontsize=18, fontweight='bold', color='black')

    # 添加灰色阴影区域
    ax.axvspan(55, 90, color='gray', alpha=0.3)
    ax.axvspan(90, 120, color='gray', alpha=0.2)

    # 设置横坐标范围和标签
    ax.set_xlim([1, 120])
    ax.set_xticks([1, 30, 60, 90, 120])
    ax.set_xticklabels(['1-Jan', '1-Feb', '1-Mar', '1-Apr', '1-May'])

    # 设置图例和标题
    ax.set_title(f'{country}', color='black', fontsize=20, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('tCO2eq/GWh')

    # 增加边框厚度，并设置纯黑色
    for spine in ax.spines.values():
        spine.set_linewidth(2)
        spine.set_color('black')

    # 设置刻度线样式
    ax.tick_params(axis='both', direction='out', width=2, colors='black', labelsize=16)
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')

    # 添加子图标签
    ax.text(0.02, 0.15, subplot_labels[i], transform=ax.transAxes, fontsize=20, fontweight='bold', color='black', ha='left', va='top')

# 移除多余的子图框架（适用于少于12个国家的情况）
for j in range(len(selected_countries), len(axes)):
    fig.delaxes(axes[j])

# 调整布局
plt.tight_layout()
plt.show()

### Calculation

In [ ]:
def calculate_constant_matrix(day, countries, other_countries, power, power_CI, storage, storage_0, flow, num_days, former_days, solution_all=None):
    # 创建空的 Series 而不是整个 DataFrame
    constant_series = pd.Series(index=countries, dtype=float)
    
    for country in countries:
        # print(country)
        # 使用 .loc 进行单步读取和赋值
        base_value = power.loc[day + former_days, country] * power_CI.loc[day + former_days, country]
        current_value = base_value
        # print(current_value)
        
        # 处理存储逻辑
        if day > 0 and solution_all is not None:
            current_value += storage.loc[day + former_days, country] * solution_all.loc[day-1, country]
        elif day == 0 and country in storage.columns:
            # 使用 .iloc 确保位置访问
            current_value += storage.loc[day + former_days, country] * storage_0.loc[1, country]
            print(storage.loc[day + former_days, country])
            # print(current_value)
        
        # 处理流量逻辑
        for other in other_countries:
            if (country, other) in flow.columns:
                current_value += flow.loc[day + former_days, (country, other)] * storage_0.loc[1, other]
        
        # 使用 .loc 进行单步赋值
        constant_series.loc[country] = current_value
    
    return constant_series

def calculate_coefficient_matrix(day, countries, load, storage, Total_export, flow, num_days, former_days):
    # 使用 numpy 数组构建矩阵更高效
    coeff_matrix = np.zeros((len(countries), len(countries)))
    
    for equation_index, country_eq in enumerate(countries):
        for country_idx, country in enumerate(countries):
            if country_idx == equation_index:
                # 主对角线元素
                value = (
                    load.loc[day + former_days, country] +
                    storage.loc[day + former_days + 1, country] +
                    Total_export.loc[day + former_days, country]
                )
                coeff_matrix[equation_index, country_idx] = value
            else:
                # 非对角线元素
                if (country_eq, country) in flow.columns:
                    coeff_matrix[equation_index, country_idx] = -flow.loc[day + former_days, (country_eq, country)]
                else:
                    coeff_matrix[equation_index, country_idx] = 0
    
    return coeff_matrix

def solve_system(coeff_matrix, constants):
    # 使用 numpy 的线性代数求解器
    solution = np.linalg.solve(coeff_matrix, constants)
    return solution

def calculate_load_CI(start_date, end_date, countries, power, power_CI, storage, storage_0, flow, load, Total_export, output_file, period, former_days):
    other_countries = list(set(power.columns) - set(countries))
    solution_all = pd.DataFrame(columns=countries)
    
    num_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days + 1

    # 补全generation_CI:
    # 确保datetime列是datetime类型
    power_CI['datetime'] = pd.to_datetime(power_CI['datetime'])
    
    # 筛选出指定日期范围内的数据
    date_mask = (power_CI['datetime'] >= pd.to_datetime(start_date)) & \
                (power_CI['datetime'] <= pd.to_datetime(end_date))
    power_CI_subset = power_CI.loc[date_mask].copy()
    
    # 补全generation_CI: 仅处理指定日期范围内的数据
    for country in countries:
        if country in power_CI_subset.columns:
            # 使用bfill填充NaN（用后方的有效值填充前方的NaN）
            power_CI_subset[country] = power_CI_subset[country].fillna(method='bfill')
            
            # 如果填充后仍有NaN（可能在末尾），使用ffill向前填充
            power_CI_subset[country] = power_CI_subset[country].fillna(method='ffill')
        else:
            print(f"Warning: Column '{country}' not found in power_CI")
    
    # 将处理后的数据合并回原始数据框
    power_CI.loc[date_mask, countries] = power_CI_subset[countries]
    
    # 第一天处理
    constant_series = calculate_constant_matrix(0, countries, other_countries, power, power_CI, storage, storage_0, flow, num_days, former_days)
    coeff_matrix = calculate_coefficient_matrix(0, countries, load, storage, Total_export, flow, num_days, former_days)
    
    solution = solve_system(coeff_matrix, constant_series.values.astype(float))
    solution_all.loc[0] = solution
    
    # 后续天处理
    for day in range(1, num_days):
        constant_series = calculate_constant_matrix(
            day, countries, other_countries, power, power_CI, storage, storage_0, flow, 
            num_days, former_days, solution_all
        )
        coeff_matrix = calculate_coefficient_matrix(day, countries, load, storage, Total_export, flow, num_days, former_days)
        
        solution = solve_system(coeff_matrix, constant_series.values.astype(float))
        solution_all.loc[day] = solution
        # print(f"Day {day}: {solution}")
    
    # 添加日期列并保存
    solution_all['date'] = pd.date_range(start=start_date, end=end_date, freq='D')
    solution_all.to_csv(output_file, index=False)
    
    return period, countries, num_days

def get_countries_for_period(period):
    if period in additional_countries:
        countries = common_countries + additional_countries[period]
        countries.sort()
        return countries
    else:
        return common_countries
    
common_countries = ['Austria', 'Belgium', 'Bulgaria', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Latvia', 'Lithuania', 'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland'] 
           

# 不同阶段的附加国家列表
additional_countries = {
    '2015-01-01 to 2015-12-31': ['United Kingdom'],
    '2016-01-01 to 2016-08-31': ['United Kingdom', 'Italy'], #, 'Cyprus'
    '2016-09-01 to 2016-12-31': ['United Kingdom', 'Italy', 'North Macedonia'], #, 'Cyprus'
    '2017-01-01 to 2017-02-28': ['United Kingdom', 'Italy', 'North Macedonia', 'Serbia'], #, 'Cyprus'
    '2017-03-01 to 2018-12-31': ['Italy', 'North Macedonia','Bosnia and Herz.', 'Serbia', 'United Kingdom'], #'Cyprus'
    '2019-01-01 to 2020-01-31': ['Bosnia and Herz.', 'Croatia', 'Italy', 'North Macedonia', 'Serbia', 'United Kingdom'], #, 'Cyprus'
    '2020-02-01 to 2021-05-31': ['Bosnia and Herz.', 'Croatia', 'Italy', 'North Macedonia', 'Moldova', 'Serbia', 'United Kingdom'],  #, 'Cyprus'
    '2021-06-01 to 2021-08-31': ['Bosnia and Herz.', 'Croatia', 'Italy', 'North Macedonia', 'Luxembourg', 'Moldova', 'Serbia'], #, 'Cyprus'
    # '2021-09-01 to 2021-12-31': ['Bosnia and Herz.', 'Croatia', 'Italy', 'North Macedonia', 'Luxembourg', 'Moldova', 'Serbia'], #, 'Cyprus' ,  'Ukraine'
    '2021-09-01 to 2021-12-31': ['Bosnia and Herz.', 'Croatia', 'Italy', 'Kosovo', 'North Macedonia', 'Luxembourg', 'Moldova', 'Serbia'], #, 'Cyprus' ,  'Ukraine'
    '2022-01-01 to 2022-03-31': ['Bosnia and Herz.', 'Croatia', 'Georgia', 'Italy', 'Kosovo', 'Luxembourg', 'Moldova', 'Serbia'], #, 'Cyprus', 'Ukraine'
    '2022-04-01 to 2022-05-31': ['Bosnia and Herz.', 'Croatia', 'Georgia', 'Italy', 'Kosovo', 'Luxembourg', 'Moldova', 'Serbia'], #, 'Cyprus'
    '2022-06-01 to 2023-12-31': ['Bosnia and Herz.', 'Croatia', 'Georgia', 'Italy', 'North Macedonia', 'Kosovo', 'Luxembourg', 'Moldova', 'Serbia'],
    '2024-01-01 to 2024-12-31': ['Bosnia and Herz.', 'Croatia', 'Georgia', 'Italy', 'North Macedonia', 'Kosovo', 'Luxembourg', 'Moldova', 'Serbia']
}

# 导入其他所需数据
power = pd.read_csv('.\daily_power_2015_2024_wide.csv',encoding='gb18030')
# 改列名Bosnia and Herz为Bosnia and Herz.
power.columns = power.columns.str.replace('Bosnia and Herz', 'Bosnia and Herz.')

power_CI = pd.read_csv('.\generation_CI_wide_nonfossilreal0.csv',encoding='gb18030')
# 改列名Bosnia and Herz为Bosnia and Herz.
power_CI.columns = power_CI.columns.str.replace('Bosnia and Herz', 'Bosnia and Herz.')

flow = pd.read_csv('.\daily_flow_2015_2024_nonredundant_noNoise.csv',header=[0,1])
# 将空值取代为0
flow.fillna(0, inplace=True)

storage = pd.read_csv('.\Storage_2015_2024_daily.csv',encoding='gb18030')

load = pd.read_csv('.\daily_load_2015_2024_noNoise.csv',encoding='gb18030')
# 将空值取代为0
load.fillna(0, inplace=True)

Total_export = pd.read_csv('.\Daily_Export.csv',encoding='gb18030')

Total_import = pd.read_csv('.\Daily_import.csv',encoding='gb18030')

storage_0 = pd.read_csv('.\Storage_0.csv',encoding='gb18030')

periods = ['2015-01-01 to 2015-12-31', '2016-01-01 to 2016-08-31', '2016-09-01 to 2016-12-31', 
           '2017-01-01 to 2017-02-28', '2017-03-01 to 2018-12-31', '2019-01-01 to 2020-01-31', '2020-02-01 to 2021-05-31', '2021-06-01 to 2021-08-31', '2021-09-01 to 2021-12-31', '2022-01-01 to 2022-03-31', 
           '2022-04-01 to 2022-05-31', '2022-06-01 to 2023-12-31', '2024-01-01 to 2024-12-31']

# period_last_time = '2022-10-01 to 2023-12-31'
for period in periods:
    # 初始化数据
    start_date = period.split(' to ')[0]
    end_date = period.split(' to ')[1]
    countries = get_countries_for_period(period)
    print(start_date)
    print(countries)
    storage_0 = storage_0  # 初始化storage_0数据

    if period != '2015-01-01 to 2015-12-31':       
        for country in countries:
            if country in countries_last_time:
                solution_last_time = pd.read_csv('.\nonfossilreal0_noNoise_load_CI_' + period_last_time + '.csv')
                storage_0[country][1] = solution_last_time[country][num_days_last_time-1]  # num_days_last_time-1
            # else:
                

    power = power # 初始化power数据
    power_CI = power_CI  # 初始化power_CI数据
    storage = storage  # 初始化storage数据
    flow = flow  # 初始化flow数据
    load = load  # 初始化load数据
    Total_export = Total_export  # 初始化Total_export数据
    # # 打印各数据的列名
    # print("power 数据的列名:", power.columns.tolist())
    # print("power_CI 数据的列名:", power_CI.columns.tolist())
    # print("storage 数据的列名:", storage.columns.tolist())
    # print("flow 数据的列名:", flow.columns.tolist())
    # print("load 数据的列名:", load.columns.tolist())
    # print("Total_export 数据的列名:", Total_export.columns.tolist())

    output_file = '.\nonfossilreal0_noNoise_load_CI_' + period + '.csv'
    # 记录2015-01-01到start_date的天数
    former_days = (pd.to_datetime(start_date) - pd.to_datetime('2015-01-01')).days
    print(former_days)

    period_last_time, countries_last_time, num_days_last_time = calculate_load_CI(start_date, end_date, countries, power, power_CI, storage, storage_0, flow, load, Total_export, output_file, period, former_days)
